In [181]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping

In [182]:
# Format data - features
boston = load_boston()
features = np.array(boston.data)

In [183]:
# Format data - target
target = np.array(boston.target)

In [184]:
# Split into training and test sets
Xtrain, Xtest, ytrain, ytest = train_test_split(features, target, test_size=0.25)

In [185]:
# Get the shape of the data
print(Xtrain.shape)
print(ytrain.shape)

n_cols = Xtrain.shape[1]

(379, 13)
(379,)


### Basic model

In [186]:
# Create model
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(n_cols,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1))

In [187]:
# Compile and fit model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(Xtrain, ytrain, validation_split=0.25)

Train on 284 samples, validate on 95 samples
Epoch 1/10
284/284 [==============================] - 1s - loss: 245.3551 - val_loss: 165.1611
Epoch 2/10
284/284 [==============================] - 0s - loss: 138.3780 - val_loss: 76.9877
Epoch 3/10
284/284 [==============================] - 0s - loss: 90.2867 - val_loss: 56.3536
Epoch 4/10
284/284 [==============================] - 0s - loss: 78.6577 - val_loss: 39.4966
Epoch 5/10
284/284 [==============================] - 0s - loss: 73.2594 - val_loss: 48.2749
Epoch 6/10
284/284 [==============================] - 0s - loss: 65.0691 - val_loss: 44.4548
Epoch 7/10
284/284 [==============================] - 0s - loss: 63.0472 - val_loss: 36.9471
Epoch 8/10
284/284 [==============================] - 0s - loss: 60.9671 - val_loss: 44.4506
Epoch 9/10
284/284 [==============================] - 0s - loss: 61.0204 - val_loss: 47.0270
Epoch 10/10
284/284 [==============================] - 0s - loss: 59.4982 - val_loss: 37.5303


In [188]:
mean_squared_error(ytest, model.predict(Xtest)[:, 0])

65.377005699694195

### Try different learning rates

Note that the SGD optimizer does not work well for continuous regression. Adam works best for regression (as shown above) while SGD works for classification (as shown in the Datacamp slides).

Further, the using the SGD optimizer + a learning rate with a Sequential model where the activation functions for all layers is 'relu' leads to a loss of nan. I've changed the activation function of the last layer to be 'tanh' to get this exercise to work. 'softmax' and 'sigmoid' also work to a certain extent. However in reality, I wouldn't use the below structure for a regression network.

In [189]:
def get_new_model():
    # Create a basic model
    n_cols = X.shape[1]
    mod = Sequential()
    mod.add(Dense(100, activation='relu', input_shape=(n_cols,)))
    mod.add(Dense(100, activation='relu'))
    # Activation would be 'relu' normally, but is 'tanh' here to get the SGD learning rate to work
    mod.add(Dense(100, activation='tanh'))
    mod.add(Dense(1))
    return(mod)

In [190]:
lr_to_test = [0.0001, 0.001, 0.01, 0.1, 0.5, 1]

for lr in lr_to_test:
    print('\nTesting model with learning rate %f' % lr)
    model2 = get_new_model()
    sgd = SGD(lr=lr)
    model2.compile(optimizer=sgd, loss='mean_squared_error')
    model2.fit(X, y)


Testing model with learning rate 0.000100
Epoch 1/10
506/506 [==============================] - 1s - loss: 340.2401     
Epoch 2/10
506/506 [==============================] - 0s - loss: 207.7320     
Epoch 3/10
506/506 [==============================] - 0s - loss: 148.3421     
Epoch 4/10
506/506 [==============================] - 0s - loss: 116.3528     
Epoch 5/10
506/506 [==============================] - 0s - loss: 99.1499      
Epoch 6/10
506/506 [==============================] - 0s - loss: 89.9163     
Epoch 7/10
506/506 [==============================] - 0s - loss: 83.6451     
Epoch 8/10
506/506 [==============================] - 0s - loss: 79.2458     
Epoch 9/10
506/506 [==============================] - 0s - loss: 76.3913     
Epoch 10/10
506/506 [==============================] - 0s - loss: 73.4567     

Testing model with learning rate 0.001000
Epoch 1/10
506/506 [==============================] - 1s - loss: 173.6382     
Epoch 2/10
506/506 [=============================

### Early stopping

In [194]:
early_stopping_monitor = EarlyStopping(patience=3)

model3 = Sequential()
model3.add(Dense(100, activation='relu', input_shape=(n_cols,)))
model3.add(Dense(100, activation='relu'))
model3.add(Dense(100, activation='relu'))
model3.add(Dense(1))

In [195]:
model3.compile(optimizer='adam', loss='mean_squared_error')
model3.fit(Xtrain, ytrain, validation_split=0.25, epochs=30, callbacks=[early_stopping_monitor])

Train on 284 samples, validate on 95 samples
Epoch 1/30
284/284 [==============================] - 1s - loss: 212.8339 - val_loss: 124.8006
Epoch 2/30
284/284 [==============================] - 0s - loss: 89.7386 - val_loss: 80.4548
Epoch 3/30
284/284 [==============================] - 0s - loss: 78.8242 - val_loss: 102.7109
Epoch 4/30
284/284 [==============================] - 0s - loss: 79.7112 - val_loss: 61.9852
Epoch 5/30
284/284 [==============================] - 0s - loss: 65.8606 - val_loss: 42.9941
Epoch 6/30
284/284 [==============================] - 0s - loss: 75.0933 - val_loss: 59.1770
Epoch 7/30
284/284 [==============================] - 0s - loss: 69.2046 - val_loss: 37.7806
Epoch 8/30
284/284 [==============================] - 0s - loss: 61.4454 - val_loss: 36.7245
Epoch 9/30
284/284 [==============================] - 0s - loss: 57.9004 - val_loss: 40.3617
Epoch 10/30
284/284 [==============================] - 0s - loss: 57.1829 - val_loss: 35.7469
Epoch 11/30
284/284 [

In [196]:
mean_squared_error(ytest, model3.predict(Xtest)[:, 0])

49.616126585894719

### More experimenting

#### More layers

In [197]:
model4 = Sequential()
model4.add(Dense(100, activation='relu', input_shape=(n_cols,)))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(1))

In [198]:
model4.compile(optimizer='adam', loss='mean_squared_error')
model4.fit(Xtrain, ytrain, validation_split=0.25, epochs=30, callbacks=[early_stopping_monitor])

Train on 284 samples, validate on 95 samples
Epoch 1/30
284/284 [==============================] - 3s - loss: 260.8121 - val_loss: 77.5534
Epoch 2/30
284/284 [==============================] - 0s - loss: 97.3663 - val_loss: 61.3699
Epoch 3/30
284/284 [==============================] - 0s - loss: 73.7777 - val_loss: 60.0684
Epoch 4/30
284/284 [==============================] - 0s - loss: 69.1082 - val_loss: 42.7861
Epoch 5/30
284/284 [==============================] - 0s - loss: 68.0020 - val_loss: 46.5539
Epoch 6/30
284/284 [==============================] - 0s - loss: 69.6800 - val_loss: 39.7008
Epoch 7/30
284/284 [==============================] - 0s - loss: 72.9710 - val_loss: 57.8890
Epoch 8/30
284/284 [==============================] - 0s - loss: 65.8712 - val_loss: 56.7154
Epoch 9/30
284/284 [==============================] - 0s - loss: 72.0773 - val_loss: 39.3544
Epoch 10/30
284/284 [==============================] - 0s - loss: 66.2463 - val_loss: 39.5148
Epoch 11/30
284/284 [==

In [200]:
mean_squared_error(ytest, model4.predict(Xtest)[:, 0])

64.667983680161015

#### More nodes

In [201]:
model5 = Sequential()
model5.add(Dense(1000, activation='relu', input_shape=(n_cols,)))
model5.add(Dense(1000, activation='relu'))
model5.add(Dense(1000, activation='relu'))
model5.add(Dense(1))

In [202]:
model5.compile(optimizer='adam', loss='mean_squared_error')
model5.fit(Xtrain, ytrain, validation_split=0.25, epochs=30, callbacks=[early_stopping_monitor])

Train on 284 samples, validate on 95 samples
Epoch 1/30
284/284 [==============================] - 2s - loss: 4195.4153 - val_loss: 515.9454
Epoch 2/30
284/284 [==============================] - 0s - loss: 244.5644 - val_loss: 175.9453
Epoch 3/30
284/284 [==============================] - 0s - loss: 95.2485 - val_loss: 48.8249
Epoch 4/30
284/284 [==============================] - 0s - loss: 70.1496 - val_loss: 53.5829
Epoch 5/30
284/284 [==============================] - 0s - loss: 67.7471 - val_loss: 44.1243
Epoch 6/30
284/284 [==============================] - 0s - loss: 67.1316 - val_loss: 47.3299
Epoch 7/30
284/284 [==============================] - 0s - loss: 63.5312 - val_loss: 45.8815
Epoch 8/30
284/284 [==============================] - 0s - loss: 62.7086 - val_loss: 41.8947
Epoch 9/30
284/284 [==============================] - 0s - loss: 61.5894 - val_loss: 37.7738
Epoch 10/30
284/284 [==============================] - 0s - loss: 65.2726 - val_loss: 42.4531
Epoch 11/30
284/284

In [203]:
mean_squared_error(ytest, model5.predict(Xtest)[:, 0])

45.810263704275414

#### More layers and nodes

In [204]:
model6 = Sequential()
model6.add(Dense(1000, activation='relu', input_shape=(n_cols,)))
model6.add(Dense(1000, activation='relu'))
model6.add(Dense(1000, activation='relu'))
model6.add(Dense(1000, activation='relu'))
model6.add(Dense(1000, activation='relu'))
model6.add(Dense(1000, activation='relu'))
model6.add(Dense(1000, activation='relu'))
model6.add(Dense(1000, activation='relu'))
model6.add(Dense(1000, activation='relu'))
model6.add(Dense(1000, activation='relu'))
model6.add(Dense(1))

In [205]:
model6.compile(optimizer='adam', loss='mean_squared_error')
model6.fit(Xtrain, ytrain, validation_split=0.25, epochs=30, callbacks=[early_stopping_monitor])

Train on 284 samples, validate on 95 samples
Epoch 1/30
284/284 [==============================] - 2s - loss: 698.3341 - val_loss: 102.0227
Epoch 2/30
284/284 [==============================] - 0s - loss: 213.9721 - val_loss: 114.0644
Epoch 3/30
284/284 [==============================] - 0s - loss: 121.5692 - val_loss: 64.2238
Epoch 4/30
284/284 [==============================] - 0s - loss: 79.3414 - val_loss: 49.0877
Epoch 5/30
284/284 [==============================] - 0s - loss: 69.0817 - val_loss: 50.4760
Epoch 6/30
284/284 [==============================] - 0s - loss: 67.2803 - val_loss: 40.8885
Epoch 7/30
284/284 [==============================] - 0s - loss: 62.6686 - val_loss: 44.8597
Epoch 8/30
284/284 [==============================] - 0s - loss: 64.0916 - val_loss: 38.8311
Epoch 9/30
284/284 [==============================] - 0s - loss: 64.1506 - val_loss: 39.8364
Epoch 10/30
284/284 [==============================] - 0s - loss: 63.9410 - val_loss: 37.9141
Epoch 11/30
284/284

In [206]:
mean_squared_error(ytest, model6.predict(Xtest)[:, 0])

67.420533055254552

#### 1,000 nodes, 2 layers

In [211]:
model7 = Sequential()
model7.add(Dense(1000, activation='relu', input_shape=(n_cols,)))
model7.add(Dense(1000, activation='relu'))
model7.add(Dense(1))

In [212]:
model7.compile(optimizer='adam', loss='mean_squared_error')
model7.fit(Xtrain, ytrain, validation_split=0.25, epochs=30, callbacks=[early_stopping_monitor])

Train on 284 samples, validate on 95 samples
Epoch 1/30
284/284 [==============================] - 1s - loss: 6515.7778 - val_loss: 4137.7961
Epoch 2/30
284/284 [==============================] - 0s - loss: 1961.2975 - val_loss: 881.8761
Epoch 3/30
284/284 [==============================] - 0s - loss: 460.8378 - val_loss: 142.7894
Epoch 4/30
284/284 [==============================] - 0s - loss: 198.9958 - val_loss: 68.8935
Epoch 5/30
284/284 [==============================] - 0s - loss: 129.1397 - val_loss: 81.3759
Epoch 6/30
284/284 [==============================] - 0s - loss: 113.0184 - val_loss: 109.7930
Epoch 7/30
284/284 [==============================] - 0s - loss: 75.3847 - val_loss: 41.9898
Epoch 8/30
284/284 [==============================] - 0s - loss: 72.8695 - val_loss: 41.6134
Epoch 9/30
284/284 [==============================] - 0s - loss: 72.8219 - val_loss: 48.8130
Epoch 10/30
284/284 [==============================] - 0s - loss: 64.7769 - val_loss: 44.6227
Epoch 11/30

In [213]:
mean_squared_error(ytest, model7.predict(Xtest)[:, 0])

52.323601616701026

### Conclusions

Based on the above, it seems that for this particular application, less layers but more nodes is optimal. Specifically, 3 layers with 1,000 nodes each produced the lowest MSE.